references
https://python.langchain.com/docs/integrations/llms/ollama/

https://api.python.langchain.com/en/latest/graph_transformers/langchain_experimental.graph_transformers.llm.LLMGraphTransformer.html

https://python.langchain.com/docs/how_to/document_loader_json/
https://thesai.org/Downloads/Volume10No10/Paper_30-JsonToOnto_Building_Owl2_Ontologies.pdf

owl to neo4j to graphRAG
https://neo4j.com/blog/using-owl-with-neo4j/?utm_source=GSearch&utm_medium=PaidSearch&utm_campaign=Evergreen&utm_content=AMS-Search-SEMCE-DSA-None-SEM-SEM-NonABM&utm_term=&utm_adgroup=DSA&gad_source=1&gclid=CjwKCAiAudG5BhAREiwAWMlSjFNimXejezc7i1kEQlYU_gi_es4y2GsJU0q_egaOoWvEB3w8YzP3SRoC-mAQAvD_BwE

https://neo4j.com/labs/neosemantics/4.0/importing-ontologies/

https://neo4j.com/developer-blog/neo4j-graphrag-workflow-langchain-langgraph/

they used microsoft graphRAG
https://medium.com/@ianormy/microsoft-graphrag-with-an-rdf-knowledge-graph-part-1-00a354afdb09


systemctl {start|stop|restart} neo4j

neo4j=1.5.25.1

In [ ]:
import networkx as nx
from langchain.chains import GraphQAChain
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph

In [2]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Sakul32524!"

graph = Neo4jGraph()

In [3]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model = "llama3.1:8b-instruct-fp16") 

llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
from langchain_core.documents import Document

text = """ 
class AlexNet(nn.Module):
    
    Neural network model consisting of layers propsed by AlexNet paper.
    
    def __init__(self, num_classes=1000):
        
        Define and allocate layers for this neural net.

        Args:
            num_classes (int): number of classes to predict with this model
        
        super().__init__()
        # input size should be : (b x 3 x 227 x 227)
        # The image in the original paper states that width and height are 224 pixels, but
        # the dimensions after first convolution layer do not lead to 55 x 55.
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),  # (b x 96 x 55 x 55)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 96 x 27 x 27)
            nn.Conv2d(96, 256, 5, padding=2),  # (b x 256 x 27 x 27)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 13 x 13)
            nn.Conv2d(256, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),  # (b x 256 x 13 x 13)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 6 x 6)
        )
        # classifier is just a name for linear layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features=(256 * 6 * 6), out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
        self.init_bias()  # initialize bias 
"""

In [9]:
documents = [Document(page_content=text)]
llm_transformer = LLMGraphTransformer(llm=llm, strict_mode=True)
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='num_classes', type='Parameter', properties={}), Node(id='num_classes=1000', type='Parameter', properties={}), Node(id='number of classes to predict with this model', type='Definition', properties={}), Node(id='Neural network model', type='Definition', properties={}), Node(id='layers proposed by AlexNet paper', type='Paper', properties={}), Node(id='(b x 3 x 227 x 227)', type='Input Size', properties={}), Node(id='AlexNet', type='Model', properties={})]
Relationships:[Relationship(source=Node(id='AlexNet', type='Model', properties={}), target=Node(id='layers proposed by AlexNet paper', type='Paper', properties={}), type='CONSISTS_OF', properties={}), Relationship(source=Node(id='AlexNet', type='Model', properties={}), target=Node(id='Neural network model', type='Definition', properties={}), type='DEFINITION', properties={}), Relationship(source=Node(id='AlexNet', type='Model', properties={}), target=Node(id='num_classes=1000', type='Parameter', properties={}), type='PARA

classes = network



In [ ]:
llm_transformer_filtered = LLMGraphTransformer(
    llm = llm,
    allowed_nodes =["Classes", "", "Parameters"],
    # allowed_relationships=[""]
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")

Nodes:[Node(id='layers propsed by AlexNet paper', type='Layers', properties={}), Node(id='AlexNet', type='Model', properties={})]
Relationships:[Relationship(source=Node(id='AlexNet', type='Model', properties={}), target=Node(id='layers propsed by AlexNet paper', type='Layers', properties={}), type='CONSISTS_OF', properties={})]


In [3]:
def query_ollama(prompt: str) -> str:
    response = llm.chat(messages=[{"role": "user", "content": prompt}])
    return response["text"]

In [4]:
graph = nx.Graph()

In [ ]:
graph_qa_chain = GraphQAChain(
    graph=NetworkxEntityGraph(graph),
    llm=query_ollama
)

In [ ]:
query = ""
result = graph_qa_chain.run(query)
print(result)